In [214]:
import pandas as pd
import numpy as np
import os
import timeit
start_time = timeit.default_timer()

In [226]:
df = pd.DataFrame()

for fname in os.listdir():
    if os.path.isfile(fname):
         if 'weekends' in fname:
            df_temp = pd.read_csv(fname)
            df_temp['weekday_or_weekend'] = 'weekend'
            df_temp['city'] = fname.split('_')[0]
            df = pd.concat([df,df_temp], axis = 0)
         elif 'weekdays' in fname:
            df_temp = pd.read_csv(fname)
            df_temp['weekday_or_weekend'] = 'weekday'
            df_temp['city'] = fname.split('_')[0]
            df = pd.concat([df,df_temp], axis = 0)
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv('airbnbprices')

In [216]:
df.drop(columns = ['lat','lng'], inplace = True)

In [217]:
numeric_features = list(df.select_dtypes(include=[np.float64]).columns)
numeric_features.pop(0)

'realSum'

In [218]:
df.multi = df.multi.astype('bool') 
df.biz = df.biz.astype('bool')
categorical_features = list(df.select_dtypes(include=[bool, object]).columns)

In [219]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_tranformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_tranformer, numeric_features),
        ('cat',categorical_transformer, categorical_features)
])

In [220]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor())
])

In [221]:
y = df.realSum
X = df.drop(columns= ['realSum'])

In [222]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [223]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['person_capacity',
                                                   'cleanliness_rating',
                                                   'guest_satisfaction_overall',
                                                   'dist', 'metro_dist',
                                                   'attr_index',
                                                   'attr_index_norm',
                                                   'rest_index',
                                                   'rest_index_norm']),
                                                 ('cat', OneHotEncoder(),
                                                  ['room_type', 'room_shared',
                                                   'room_private',
                                                   'host_is_superhost', 'multi',
                      

In [224]:
from sklearn.metrics import mean_absolute_error, mean_squared_error   

predictions = pipeline.predict(X_test)

print("mean absolute error: %.3f" % mean_absolute_error(predictions, y_test))
elapsed = timeit.default_timer() - start_time

mean absolute error: 61.853


In [225]:
elapsed

56.90930910000043

In [227]:
df

,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,...,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,weekday_or_weekend,city
0,194.033698,Private room,False,True,2.0,False,1,0,10.0,93.0,...,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,weekday,amsterdam
1,344.245776,Private room,False,True,4.0,False,0,0,8.0,85.0,...,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,weekday,amsterdam
2,264.101422,Private room,False,True,2.0,False,0,1,9.0,87.0,...,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,weekday,amsterdam
3,433.529398,Private room,False,True,4.0,False,0,1,9.0,90.0,...,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,weekday,amsterdam
4,485.552926,Private room,False,True,2.0,True,0,0,10.0,98.0,...,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,weekday,amsterdam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51702,715.938574,Entire home/apt,False,False,6.0,False,0,1,10.0,100.0,...,0.530181,0.135447,219.402478,15.712158,438.756874,10.604584,16.37940,48.21136,weekend,vienna
51703,304.793960,Entire home/apt,False,False,2.0,False,0,0,8.0,86.0,...,0.810205,0.100839,204.970121,14.678608,342.182813,8.270427,16.38070,48.20296,weekend,vienna
51704,637.168969,Entire home/apt,False,False,2.0,False,0,0,10.0,93.0,...,0.994051,0.202539,169.073402,12.107921,282.296424,6.822996,16.38568,48.20460,weekend,vienna
51705,301.054157,Private room,False,True,2.0,False,0,0,10.0,87.0,...,3.044100,0.287435,109.236574,7.822803,158.563398,3.832416,16.34100,48.19200,weekend,vienna
